# Carhart Factor Model: Understanding the Factors #

In [26]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from data_downloader import get_market_data
from capm_toolkit import capm_regression
from other_data_functions import rolling_calc_rstr
from portfolios_toolkit import calculate_analytics

In [2]:
# We can download the premiums in the Fama and French website
momemtum_df = pd.read_csv(r'..\additional_data\famafrench_momentum.csv')
momemtum_df.set_index('Date', inplace=True)
momemtum_df.index = pd.to_datetime(momemtum_df.index)
momemtum_df = momemtum_df.div(100)
momemtum_df.columns = ['momentum']

momemtum_df

In [3]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(momemtum_df['momentum'].cumsum(), label='Momentum (by Fama and French)', alpha=1)

# Config
plt.title('Cumulative MOM Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [4]:
momentum_etf = get_market_data(
    ticker='MTUM', 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

momentum_etf

In [5]:
# Calling Benchmark
benchmark = get_market_data(
    ticker='^GSPC', 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

benchmark

In [6]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(momentum_etf['returns'].cumsum(), label='Momentum iShares ETF', alpha=1)
plt.plot(benchmark['returns'].cumsum(), label='S&P 500', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [7]:
# Calculate Differential
differential = momentum_etf['returns'] - benchmark['returns']

# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(differential.cumsum(), label='Differential', alpha=1)
plt.plot(momemtum_df['momentum'].cumsum(), label='Momentum (by Fama and French)', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

### Othogonality ###

In [8]:
# We will use a linear regression to orthogonalize momentum
model = capm_regression(
    momentum_etf['returns'],
    benchmark['returns'],
    window = len(momentum_etf['returns']),
    WLS=True
)

print(model.summary())

In [9]:
# Residuals do not contain the Market Effects
residuals = model.resid
residuals.name = 'residuals'

residuals

In [10]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(differential.cumsum(), label='Differential', alpha=1)
plt.plot(residuals.cumsum(), label='Orthogonalized ETF', alpha=1)
plt.plot(momemtum_df['momentum'].loc[:'2024-12'].cumsum(), label='Momentum (by Fama and French)', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [11]:
# Check correlations:
wml_correlation = residuals.corr(momemtum_df['momentum'].loc[:'2024-12'])

wml_correlation

In [12]:
returns_df = pd.read_csv(r'..\additional_data\stocks_returns.csv')
returns_df = returns_df.rename(columns={'Unnamed: 0': 'Date'})
returns_df.set_index('Date', inplace=True)
returns_df.index = pd.to_datetime(returns_df.index)

returns_df

In [13]:
# Create the RS DFs
relative_strenght_long = rolling_calc_rstr(
    returns_df,
    window_size=252,
    half_life=126
).T

relative_strenght_short = rolling_calc_rstr(
    returns_df,
    window_size=28,
    half_life=14,
    min_obs=13
).T

In [14]:
relative_strenght = (relative_strenght_long - relative_strenght_short).dropna()

relative_strenght

In [15]:
# Define the Decomposition Function
def momentum_decomposition(
    returns_df, 
    momentum_df
):
    # Common Indexes
    common_index = returns_df.index.intersection(momentum_df.index)
    
    # Reindex
    returns_df = returns_df.loc[common_index]
    momentum_df = momentum_df.loc[common_index]

    # Initialize lists to store portfolio returns
    winner_list, neutral_list, loser_list = [], [], []
    
    # Get unique quarters
    months = sorted(set([date.to_period('M') for date in common_index]))
    
    # Dictionary to store quarterly classifications and weights
    monthly_classifications = {}

    # Classification and Weights
    for month in months:
        # Select only the last available date of the quarter
        month_dates = [date for date in common_index if date.to_period('M') == month]
        rebalance_date = month_dates[-1]  # Last day of the quarter
        
        # Momentum Factor for rebalance date
        momentum_factor_df = pd.DataFrame([momentum_df.loc[rebalance_date]], index=['mom']).T.dropna()
        
        # Classify stocks into Low, Neutral, and High based on quantiles
        lower = momentum_factor_df['mom'].quantile(0.3)
        upper = momentum_factor_df['mom'].quantile(0.7)

        momentum_factor_df['momentum_class'] = 'neutral'
        momentum_factor_df.loc[momentum_factor_df['mom'] <= lower, 'momentum_class'] = 'loser'
        momentum_factor_df.loc[momentum_factor_df['mom'] >= upper, 'momentum_class'] = 'winner'
        
        # Target Data
        r_df = pd.DataFrame([returns_df.loc[rebalance_date]], index=['returns']).T
        
        # Assign market caps to value classes
        losers = r_df.loc[momentum_factor_df[momentum_factor_df['momentum_class'] == 'loser'].index]
        neutrals = r_df.loc[momentum_factor_df[momentum_factor_df['momentum_class'] == 'neutral'].index]
        winners = r_df.loc[momentum_factor_df[momentum_factor_df['momentum_class'] == 'winner'].index]
        
        # Compute weights
        loser_weights = pd.Series(1 / len(losers), index=losers.index)
        neutral_weights = pd.Series(1 / len(neutrals), index=neutrals.index)
        winner_weights = pd.Series(1 / len(winners), index=winners.index)
        
        # Store classifications and weights
        monthly_classifications[month] = {
            "loser": loser_weights,
            "neutral": neutral_weights,
            "winner": winner_weights
        }
    
    # Iterate over all available dates to compute daily returns
    for date in common_index:
        month_key = date.to_period('M')  # Get quarter of the current date
        
        if month_key in monthly_classifications:
            # Retrieve stored classification and weights
            loser_weights = monthly_classifications[month_key]["loser"]
            neutral_weights = monthly_classifications[month_key]["neutral"]
            winner_weights = monthly_classifications[month_key]["winner"]
            
            # Retrieve daily returns
            target = pd.DataFrame([returns_df.loc[date]], index=['returns']).T
            
            loser_returns = target.reindex(loser_weights.index).dropna()
            neutral_returns = target.reindex(neutral_weights.index).dropna()
            winner_returns = target.reindex(winner_weights.index).dropna()
            
            # Compute portfolio returns
            loser_result = loser_weights.reindex(loser_returns.index).T @ loser_returns
            neutral_result = neutral_weights.reindex(neutral_returns.index).T @ neutral_returns
            winner_result = winner_weights.reindex(winner_returns.index).T @ winner_returns
            
            # Store results
            loser_list.append(loser_result.values[0] if not loser_result.empty else None)
            neutral_list.append(neutral_result.values[0] if not neutral_result.empty else None)
            winner_list.append(winner_result.values[0] if not winner_result.empty else None)

    # Create final DataFrame
    momentum_portfolios = pd.DataFrame({
        'winner': winner_list,
        'neutral': neutral_list,
        'loser': loser_list
    }, index=common_index)
    
    return momentum_portfolios

In [16]:
# Create DataFrames
momentum_portfolios_returns = momentum_decomposition(returns_df, relative_strenght)

momentum_portfolios_returns

In [17]:
# Calculate the Analytics
mom_portfolios_analytics = calculate_analytics(momentum_portfolios_returns)

mom_portfolios_analytics

In [18]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(momentum_portfolios_returns.cumsum(), label=momentum_portfolios_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns (Momentum Adjusted) Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [19]:
# Calculate the WML Premium
WML = momentum_portfolios_returns['winner'] - momentum_portfolios_returns['loser']

In [20]:
# Plot
plt.figure(figsize=(10, 6))
plt.plot(WML.cumsum(), label='WML Premium', color = 'salmon', alpha=1)

# Config
plt.title('WML Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [21]:
model = capm_regression(
    WML,
    benchmark['returns'].loc['2016':],
    window = len(WML),
    WLS=True
)

print(model.summary())

In [22]:
# Calculate Residuals
mom_residuals = model.resid
mom_residuals.name = 'mom_residuals'

mom_residuals

In [28]:
plt.figure(figsize=(10, 6))
plt.plot(mom_residuals.cumsum(), label='WML Premium', alpha=1)
plt.plot(momemtum_df['momentum'].loc['2016':'2024'].cumsum(), label='Momentum (by Fama and French)', alpha=1)

# Config
plt.title('WML Orthogonalized Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [24]:
# Check the Correlation
mom_residuals.corr(momemtum_df['momentum'].loc['2016':'2024'])

In [27]:
# Data
x = mom_residuals
y = momemtum_df['momentum'].loc['2016':'2024']

# Fit a linear regression (slope and intercept)
slope, intercept = np.polyfit(x, y, 1)

# Create points for the trendline
x_fit = np.linspace(min(x), max(x), 100)
y_fit = slope * x_fit + intercept

# Plot
fig, ax1 = plt.subplots(dpi=600)

plt.scatter(x, y, label='Data')
plt.plot(x_fit, y_fit, color='red', label='Trendline')

plt.ylabel('WML (by F&F)')
plt.xlabel('WML (Using Portfolios)')
plt.legend()
plt.show()